Naive Bayes Tutorial Predicting survival from titanic crash

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [4]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [5]:
#inputs.Sex = inputs.Sex.map({'male': 1, 'female': 2})

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


**I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female**

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)

In [18]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [25]:
model.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [26]:
model.score(X_test,y_test)

0.8212290502793296

In [27]:
X_test[0:10]

,Pclass,Age,Fare,female
615,2,24.0,65.0000,1
320,3,22.0,7.2500,0
823,3,27.0,12.4750,1
741,1,36.0,78.8500,0
105,3,28.0,7.8958,0
771,3,48.0,7.8542,0
683,3,14.0,46.9000,0
275,1,63.0,77.9583,1
592,3,47.0,7.2500,0
174,1,56.0,30.6958,0


In [28]:
y_test[0:10]

615    1
320    0
823    1
741    0
105    0
771    0
683    0
275    1
592    0
174    0
Name: Survived, dtype: int64

In [29]:
model.predict(X_test[0:10])

array([1, 0, 1, 0, 0, 0, 0, 1, 0, 0], dtype=int64)

In [30]:
model.predict_proba(X_test[:10])

array([[0.12163853, 0.87836147],
       [0.95163168, 0.04836832],
       [0.40098743, 0.59901257],
       [0.52256541, 0.47743459],
       [0.95722404, 0.04277596],
       [0.95948785, 0.04051215],
       [0.91922179, 0.08077821],
       [0.02331776, 0.97668224],
       [0.95976822, 0.04023178],
       [0.76231192, 0.23768808]])

**Calculate the score using cross validation**

In [40]:
from sklearn.model_selection import cross_val_score
a= cross_val_score(GaussianNB(),X_train, y_train, cv=5)
a

array([0.76923077, 0.74825175, 0.78321678, 0.76760563, 0.78723404])

In [38]:
cross_val_score

<function sklearn.model_selection._validation.cross_val_score(estimator, X, y=None, groups=None, scoring=None, cv='warn', n_jobs=None, verbose=0, fit_params=None, pre_dispatch='2*n_jobs', error_score='raise-deprecating')>

In [41]:
a.mean()

0.7711077954110619